# 뉴스 데이터
뉴스에는 사람들의 이야기, 생각, 행동에 관하여 많은 가치 있는 정보가 들어있다. 사람들은 뉴스 기사를 분석하여 [주식 시장을 예측](http://people.kth.se/~gyozo/docs/financial-prediction.pdf)하기도 하고, 뉴스를 분석하여 관련 있는 뉴스들을 자동으로 [요약하기](http://dl.acm.org/citation.cfm?id=1289212)도 하며, 트위터의 감성을 [분석하기도](http://dl.acm.org/citation.cfm?id=1487024)한다. 또한 작가에 대한 정치적인 성향을 [분석](https://pdfs.semanticscholar.org/3b46/d2a5ac4d26ce7f62c724663b6ee282471764.pdf)하기도 하며 관련된 뉴스나 분석을 [찾기도](http://www.aaai.org/Papers/Symposia/Spring/2006/SS-06-03/SS06-03-016.pdf)한다.

이번 예제에서는 구글의 [경제 뉴스](https://www.google.com/finance/market_news) 데이터를 수집하여 [애플](http://www.apple.com/) 사에 대한 정보를 얻는 프로그램을 작성하겠다. 

구글의 경제 뉴스를 읽는데 편리한 API가 따로 존재하지는 않는다. 여기서는 해당 사이트의 소스 코드를 분석하여 관련된 부분의 데이터를 자동으로 추출하는 라이브러리를 사용하겠다.

애플사 "AAPL"와 관련된 경제 뉴스를 얻기 위해서 애플사의 프로파일 정보을 아래의 사이트에서 찾을 수 있다.

 https://www.google.com/finance?q=NASDAQ:AAPL&ei=LlMtWILeNJLNuATF7JawBQ
 
여기서 "news"를 선택하면 관련 뉴스를 볼 수 있다 (아래 링크 선택)  https://www.google.com/finance/company_news?q=NASDAQ%3AAAPL&ei=slUtWOmSJoveugStsJn4Dw

참고로 구글에서 "ei google search parameter"를 [검색하면](http://web.archive.org/web/20140625145138/http://www.rankpanel.com/blog/google-search-parameters/) 다음과 같은 [정보를](https://deedpolloffice.com/blog/articles/decoding-ei-parameter) 얻는다. 여기서 ei 파라미터는 다음을 기술한다 (1) 이 링크를 찾은 URL 주소 (여기서는 애플사 AAPL의 경제 뉴스 사이트. (2) 이 링크를 선택한 날짜와 시간. 따라서 &ei 이하의 파라미터는 무시하고 진행해도 되며 다음과 같은 주소를 사용하면 된다. https://www.google.com/finance/company_news?q=NASDAQ%3AAAPL

만일 애플사가 아닌 다른 회사의 정보를 얻고 싶으면 다른 회사의 주식시장 기호를 사용하면 된다. 예를 들어 구글사이면 "AAPL"대신 "GOOGL"을 사용하면 된다.

이러한 작업은 사람이 하는 경우에는 직관적으로 쉽게 찾고 바꿀 수 있지만, 컴퓨터 프로그램이 이를 대시 수행하게 하기에는 어렵다.

<p style="text-align:center;"><img src=./files/aapl_news.png alt="AAPL news on Google Finance News. Screenshotted with Gyazo." style="width: 60%"></p>

만일 이러한 웹페이지를 구성하는 코드의 구조를 알고 있다면 프로그램에서 어떤 부분을 조사하도록 지정하는 것이 쉬울 것이다. 웹 페이지의 코드를 파악하려면 크롬의 경우 웹페이지에서 마우스 우측 클릭을 하고 "검사(Inspect)"를 선택하면 해당 웹페이지의 내용을 상세하게 볼 수 있다. 궁금한 곳 위에서 우측 클릭을 하며 해당 부분을 명시적으로 강조하여 보여준다. 예를 들어 아래는 뉴스 기사를 보여주는 영역에 대한 소스 코드를 보여준다.

<p style="text-align:center;"><img src=./files/aapl_news_source_code.png alt="AAPL news on Google Finance News. Screenshotted with Gyazo." style="width: 60%"></p>

이 그림을 보면, 각 뉴스 기사는 독립적인 섹션으로 구성된 것처럼 보이는 것을 알 수 있다. 이 그림에서 선택된 섹션은 "g-section news sfe-break-bottom-16"라고 표현되어 있다.

<p style="text-align:center;"><img src=./files/aapl_news_source_for_article.png alt="AAPL news on Google Finance News. Screenshotted with Gyazo." style="width: 60%"></p>

이제 이 웹사이트 정보로부터 어떻게 섹션 정보를 얻고, 해당 URL을 얻고, 이 URL로부터 뉴스 기사를 텍스트로 얻을지를 알아보자. 다행히 파이선은 이러한 작업을 수행하기 위해서 매우 편리한 세가지 라이브러리를 다음과 같이 제공한다.


[Requests](http://docs.python-requests.org/en/master/), [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) 그리고 [Newspaper](http://newspaper.readthedocs.io/en/latest/)이며 이들은 아래와 같은 명령문으로 설치할 수 있다.

 `pip3 install requests`, 
 `pip3 install bs4`,
 `pip3 install newspaper3k`.

###### 데이터 소스
뉴스드을 포함한 데이터를 아래의 사이트에서 구할 수 있다.

- https://www.google.com/finance/company_news?q=NASDAQ%3AAAPL

###### 1 단계 : 데이터 저장소 확인
데이터를 저장할 곳을 확인한다.

In [1]:
# 아래에서 '.'는 현재 작업 디렉토리를 가리킨다 (즉, 현재 프로그램이 저장되어 있는곳을 말한다)
# 이렇게 현재 디렉토리를 중심으로 프로그램을 작성함으로써 프로그램에서 절대 경로를 다루지 않아도 되어 편리하다
# 아래는 고정된 이름의 디렉토리를 정의하는데 이와 같이 상수화된 변수는 프로그램의 첫부분에서 정의하는 것이 일반적이다

path_to_directory = "./data"
path_to_data = "./data/news"

# 아래에서 os 는 운영체제와 통신하기 위해 사용하는 라이브러리이다
# 이 라이브러리를 사용하면 운영 체제에게 임의의 명령문을 내릴 수 있다

import os  

if not os.path.isdir(path_to_directory): # if there is no data directory, create one
    os.mkdir(path_to_directory)

# shutil은 파일과 디렉터리를 다루는 유틸리티이다
# 기존에 같은 이름의 디렉토리가 존재하는지 확인하고 그 내용을 비우는 작업을 한다
# 이렇게 하는 목적은 새로운 데이터셋을 안전하게 사용하기 위해서이다

import shutil
    
if os.path.isdir(path_to_data):
    shutil.rmtree(path_to_data) # 디렉토리와 그 내용을 모두 지운다

os.mkdir(path_to_data)

###### 2 단계 : 데이터 다운로드
이제 데이터를 다운로드한다. 이를 위해 BeautifulSoup과 newspaper를 사용하고, 파이선 라이브러리 pickle을 이용하여 데이터를 자신의 컴퓨터에 저장한다.

In [2]:
# requests는 URL을 다루기 위해 사용한다 
# BeautifulSoup 는 요청 응답으로 받은 데이터를 분석(parse)하기 위해 사용한다
# newspaper는 뉴스 기사의 url을 해석하여 읽기에 편리한 문서 형태 Article 객체로 변환한다

import requests
from bs4 import BeautifulSoup
import newspaper

# 여기서는 목적지 URL 주소를 직접 입력했다
# 만일 입력할 목적지 주소가 많다면 Requests나 BeautifulSoup4 라이브러리를 사용하여
# 웹 사이트에서 자동으로 링크 주소를 얻는 방법도 있다

# 주의: 데이터 수집을 너무 자주 하지 않도록 해야 한다
# 데이터 수집이 너무 빈번하면 구글에서 일시적으로 이 IP주소에서 하는 접근을 막을 수도 있다
# 얼마나 자주 데이터를 수집해도 되는지에 대해서는 아래의 글을 참조하기 바란다
# https://news.ycombinator.com/item?id=360233

# 만일 더 많은 뉴스를 수집하려면
# URL에 다음과 같은 옵션 파라미터를 추가할 수 있다 &start=0&num=1000 

# 만일 데이터 수집을 더 빠르게 수행하려면, 다수의 데이터 요청을 여러 IP 주소에서 동시에
# 수행하는 방법이 있다
# 여러대의 컴퓨터를 사용하거나 또는 한 컴퓨터에서 멀티쓰레딩이나 멀이 프로세싱을
# 수행하는 방법도 있다.

url = "https://www.google.com/finance/company_news?q=NASDAQ%3AAAPL&start=0&num=100"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

urls = []
# 아래에서 사용하는 데이터 구조는 현재 섭비스 되는 내용을 기준으로 작성한 것이다
# 혹시 데이터 구조가 변경되면 아래 코드를 변경해야 한다

sections = soup.findAll("div", {"class": "g-section news sfe-break-bottom-16"})

# sections에 들어 있는 URL을 찾아낸다
for section in sections:
    urls += [url.get('href') for url in section.find_all('a')]

# Google의 경제 관련 URL만 골라낸다
urls = [url for url in urls if url[:len("http://news.google.com/news/url?")] == "http://news.google.com/news/url?"]

# 링크 주소에서 필요한 부분만 추출한다
urls = [url.split("&url=")[1].split("&cid")[0] for url in urls]

# 뉴스 기사를 번역하여 Newspaper 객체로 변환한다
# 이 작업이 항상 성공적으로 수행된는 것은 아니므로 오류를 대비하여 exception을 처리해야 한다

articles = []
for url in urls:
    try:
        article = newspaper.Article(url)
        article.download()
        article.parse()
        if len(article.text) >= 1:
            articles = articles + [article]
        print("Downloaded {} successfully.".format(url))
    except Exception as e:
        print("Downloading {} failed with exception {}".format(url, e))


# 뉴스 기사를 다운로드하여 파일로 저장한다
# pickle은 임의의 파이선 객체를 저장하고 로드하는 라이브러리이다
# Python 2에서는 두가지 Pickle 버전이 있었다: pickle과 cPickle (개선된 버전)
# Python 3에서는 이것이 pickle과 _pickle로 변경되었다
# 자동으로 import되며 최적의 버전을 불러서 사용한다
# 자세한 내용은 다음을 참조하기 바란다 https://docs.python.org/3.1/whatsnew/3.0.html#library-changes

import pickle

name = "news.p"
path = os.path.join(path_to_data, name)

with open(path, "wb") as outfile: # (w)rite (b)ytes
    pickle.dump(articles, outfile)

print("Finished saving news articles.")

Downloaded http://www.foxbusiness.com/markets/2016/11/17/1-awesome-new-feature-coming-to-apple-inc-iphone-8.html successfully.
Downloaded http://siliconangle.com/blog/2016/11/17/apple-reduces-repair-cost-for-iphone-6-plus-touch-disease-denies-responsibility/ successfully.
Downloaded http://www.nasdaq.com/article/apple-inc-reportedly-considering-making-iphones-in-america-cm710908 successfully.
Downloaded http://www.fool.com/investing/2016/11/17/apple-incs-strange-fascination-with-original-conte.aspx successfully.
Downloaded http://seekingalpha.com/article/4024334-xiaomi-builds-smartphone-screen-apple-built successfully.
Downloaded http://www.fool.com/investing/2016/11/17/1-apple-inc-macbook-pro-complaint-that-makes-no-se.aspx successfully.
Downloaded http://investorplace.com/2016/11/apple-inc-aapl-new-macbook-pro-iplace/ successfully.
Downloaded https://www.cnet.com/news/apples-says-you-need-macbook-pro-to-have-truly-great-ideas/ successfully.
Downloaded http://www.fool.com/investing/20

###### 3 단계 : 로드 데이터
이제 pickled 객체를 로드하면 된다

In [3]:
data = None
with open(path, 'rb') as infile:# (r)ead (b)ytes
    data = pickle.load(infile)

###### 4 단계 : 데이터 체크
각 뉴스 글은 최소한 한 바이트 이상의 내용을 담고 있어야 한다

In [4]:
for article in articles:
    assert(len(article.text) >= 1)

이제 데이터를 다운로드하고,저장하고,내용을 검증하였다.

###### 5 단계 : 코드 재구성

지금까지 작성된 코드를 하나로 재구성하였다. load_news.py 파일로 만들고 메인 함수는 아래와 같이 호출하면 된다.

`data = load_news()`

In [6]:
import os
import requests
from bs4 import BeautifulSoup
import newspaper
import pickle

path_to_directory = "./data"
path_to_data = os.path.join(path_to_directory, "news")

name = "news.p"
path = os.path.join(path_to_data, name)

def check_destination():
    if not os.path.isdir(path_to_directory):
        os.mkdir(path_to_directory)
    
    if not os.path.isdir(path_to_data):
        os.mkdir(path_to_data)

def download_data(verbose=False): 
    r = requests.get("https://www.google.com/finance/company_news?q=NASDAQ%3AAAPL")
    soup = BeautifulSoup(r.text, 'html.parser')

    urls = []
    
    sections = soup.findAll("div", {"class": "g-section news sfe-break-bottom-16"})
    for section in sections:
        urls += [url.get('href') for url in section.find_all('a')]

    urls = [url for url in urls if url[:len("http://news.google.com/news/url?")] == "http://news.google.com/news/url?"]
    urls = [url.split("&url=")[1].split("&cid")[0] for url in urls]

    articles = []
    for url in urls:
        try:
            article = newspaper.Article(url)
            article.download()
            article.parse()
            if len(article.text) >= 1:
                articles = articles + [article]
            if verbose:
                print("Downloaded {} successfully.".format(url))
        except Exception as e:
            if verbose:
                print("Downloading {} failed with exception {}".format(url, e))

    with open(path, "wb") as outfile: # (w)rite (b)ytes
        pickle.dump(articles, outfile)

    print("Finished saving news articles.")
    
def verify_data(data):
    for article in data:
        assert(len(article.text) >= 1)
            
def load_news(overwrite=False):
    if overwrite or not os.path.exists(path):
        check_destination()
        download_data(verbose=False)
        
    data = None
    with open(path, 'rb') as infile:
        data = pickle.load(infile)
    verify_data(data)
    
    return data

In [8]:
data = load_news()
print(data)

Finished saving news articles.
[<newspaper.article.Article object at 0x112b05f28>, <newspaper.article.Article object at 0x10e2c3cc0>, <newspaper.article.Article object at 0x1043b7080>, <newspaper.article.Article object at 0x10e29a630>, <newspaper.article.Article object at 0x10e2c8860>, <newspaper.article.Article object at 0x10e2c85f8>, <newspaper.article.Article object at 0x1120e98d0>, <newspaper.article.Article object at 0x1120e9a58>, <newspaper.article.Article object at 0x1120f5208>, <newspaper.article.Article object at 0x1120f56d8>, <newspaper.article.Article object at 0x112b2b748>, <newspaper.article.Article object at 0x10e2ca860>, <newspaper.article.Article object at 0x10e2ca1d0>, <newspaper.article.Article object at 0x10e2c9cf8>, <newspaper.article.Article object at 0x10e2c98d0>, <newspaper.article.Article object at 0x10e2cb048>, <newspaper.article.Article object at 0x10e2cb7f0>, <newspaper.article.Article object at 0x10e2cba90>, <newspaper.article.Article object at 0x112b0a0f0>,